Survey
====

Plots and tables for survey responses

In [1]:
import os

%load_ext autoreload
%autoreload 2

img_dir = "outputs/imgs/survey/"
if not os.path.exists(img_dir):
    os.makedirs(img_dir)

In [2]:
"""
First let's find out the reasons that people might have rejected it or who responded "2" (no) to the smartwatch question 
Also find the number of people who responded "1" (yes), so we can find the total number who were asked

"""

import pandas as pd
from analysis_utils import read

all_df = read.survey_info()

# If it's empty that's effectively not rejecting
reject_smartwatch = all_df["field_note"].str.contains(
    "reject smart", case=False, na=False
)

survey_no = (all_df["smart1_10to17"] == 2) | (all_df["smart1_7to9"] == 2)

reject = reject_smartwatch | survey_no
accept = (all_df["smart1_10to17"] == 1) | (all_df["smart1_7to9"] == 1)

all_df.loc[
    reject, ["residents_id", "smart1_7to9", "smart1_10to17", "field_note"]
].reset_index()

# Check no one is in both groups
assert not (reject & accept).any()

# print(f"N reject: {reject.sum()}; N accept {accept.sum()}")
# print(
#     f"One of these acceptances should be removed, because: {all_df[all_df['residents_id'] == 15671]['field_note'].values[0]}"
# )

In [ ]:
"""
Show who accepted the smartwatch

"""

display_cols = [
    "smart1_7to9",
    "smart1_10to17",
    "field_note",
    "smart2a_10to17",
    "smart2a_7to9",
    "smart2_10to17",
]

tmp = (
    all_df.loc[accept, display_cols]
    .groupby(display_cols[:2], dropna=False)
    .size()
    .reset_index(name="Count")
)
print(f"{tmp['Count'].sum()}, but one of these deleted the EMA app")
tmp

In [ ]:
"""
Show the ones who rejected the smartwatch outright

"""

tmp = (
    all_df.loc[reject, display_cols]
    .groupby(display_cols, dropna=False)
    .size()
    .reset_index(name="Count")
)
print(tmp["Count"].sum(), "rejected the smartwatch; reasons in `tmp`")

In [5]:
"""
Show the ones who accepted the smartwatch

"""

tmp = all_df.loc[accept, display_cols]
counts = tmp.groupby(display_cols, dropna=False).size().reset_index(name="Count")

In [6]:
"""
Read in the questionnaire data

"""

from analysis_utils import clean

qnaire_data = clean.cleaned_survey()

In [ ]:
"""
Find the number of each who agreed to take part in the smartwatch study

"""

import pandas as pd

# We only care about the participants who were asked about the smartwatches
child_df = qnaire_data[qnaire_data["child"] & ~pd.isna(qnaire_data["smart1_7to9"])]
adolescent_df = qnaire_data[
    qnaire_data["adolescent"] & ~pd.isna(qnaire_data["smart1_10to17"])
]

child_df["smart1_7to9"].value_counts(dropna=False), adolescent_df[
    "smart1_10to17"
].value_counts(dropna=False)

In [ ]:
"""
For the participants who didn't take part, find out why

"""

# print("Child reasons for not taking part in the study:")
# 
# print(child_df[child_df["smart1_7to9"] == 2]["smart2_7to9"].value_counts(dropna=False))
# 
# # Here, 7 means "Other"
# print(
#     child_df[(child_df["smart1_7to9"] == 2) & (child_df["smart2_7to9"] == 7)][
#         ["smart1_7to9", "smart2a_7to9"]
#     ]
# )

In [9]:
# print("Adolescent reasons for not taking part in the study:")
# print(
#     adolescent_df[adolescent_df["smart1_10to17"] == 2]["smart2_10to17"].value_counts(
#         dropna=False
#     )
# )
# 
# # Here, 7 means "Other" and -9 means "Refused to answer"
# print("\nother reasons:\n")
# adolescent_df[(adolescent_df["smart1_10to17"] == 2)][
#     ["smart1_10to17", "smart2_10to17", "smart2a_10to17", "field_note"]
# ]

In [ ]:
""" Keep only the ones who agreed to take part in the smartwatch study """

adolescent_df = adolescent_df[adolescent_df["smart1_10to17"] == 1]
child_df = child_df[child_df["smart1_7to9"] == 1]

len(adolescent_df), len(child_df)

In [ ]:
"""
Make a bar chart of experience of using the smartwatch

"""

import matplotlib.pyplot as plt

child_counts = child_df["smart3_7to9"].value_counts(dropna=False)
adolescent_counts = adolescent_df["smart3_10to17"].value_counts(dropna=False)

# Replace the NaN in the index with 0
child_counts.index = child_counts.index.fillna(0)
adolescent_counts.index = adolescent_counts.index.fillna(0)

fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)

axes[0].bar(child_counts.index, child_counts.values)
axes[1].bar(adolescent_counts.index, adolescent_counts.values, color="C1")

# Label each bar with the number
for i, v in child_counts.items():
    axes[0].text(i, v + 0.5, str(v), ha="center")
for i, v in adolescent_counts.items():
    axes[1].text(i, v + 0.5, str(v), ha="center")


for axis in axes:
    axis.set_xlabel("Experience of using smartwatch")
    axis.set_xticks(range(6), ["N/A", *range(1, 6)])

axes[0].set_title("Children")
axes[0].set_ylabel("Count")
axes[1].set_title("Adolescents")

fig.suptitle("How would you rate your child's/your experience of using the smartwatch?")
fig.tight_layout()
child_counts

fig.savefig(f"{img_dir}experience.png")

In [ ]:
""" Plot combined """

child_counts = child_df["smart3_7to9"].value_counts(dropna=False)
adolescent_counts = adolescent_df["smart3_10to17"].value_counts(dropna=False)

all_counts = child_counts.add(adolescent_counts, fill_value=0)

# Replace the NaN in the index with 0
all_counts.index = all_counts.index.fillna(0)
display(all_counts)


fig, axis = plt.subplots(figsize=(10, 5))

axis.bar(all_counts.index, all_counts.values)

# Label each bar with the number
# for i, v in all_counts.items():
#     axis.text(i, v + 0.5, str(int(v)), ha="center")


axis.set_xlabel("Experience of using smartwatch")
axis.set_xticks(range(6), ["N/A", *range(1, 6)])
# axis.set_ylabel("Count")

# Change x axis labels
labels = ["No Response", "1\nDid not like it at all", *range(2, 5), "5\nReally liked it"]
axis.set_xticklabels(labels)

fig.suptitle("How would you rate your child's/your experience of using the smartwatch?")
fig.tight_layout()

fig.savefig(f"{img_dir}all_experience.png")

In [ ]:
"""
Would you do it again?

"""

fig, axis = plt.subplots(figsize=(10, 5))

child_counts = child_df["smart4_7to9"].value_counts(dropna=False)
adolescent_counts = adolescent_df["smart4_10to17"].value_counts(dropna=False)

# Fill NA with a negative value so it appears at the bottom of my plot
na_val = -100
child_counts.index = child_counts.index.fillna(na_val)
adolescent_counts.index = adolescent_counts.index.fillna(na_val)

child_counts.sort_index(inplace=True)
adolescent_counts.sort_index(inplace=True)

child_counts, adolescent_counts
# Stacked bar chart
color = {na_val: "k", 1.0: "g", 2.0: "gold", 3.0: "r", -9.0: "grey"}

total = 0
for val, count in child_counts.items():
    axis.bar(0, count, color=color[val], label=i, bottom=total, width=0.4)
    total += count

total = 0
for val, count in adolescent_counts.items():
    axis.bar(1, count, color=color[val], label=i, bottom=total, width=0.4)
    total += count

axis.legend(
    handles=[plt.Rectangle((0, 0), 1, 1, color=color[val]) for val in color],
    labels=["N/A", "Yes", "Maybe", "No", "Refused to Answer"],
)

axis.set_xticks([0, 1], ["Children", "Adolescents"])

axis.set_title(
    "If your child/you were asked to use the smartwatch again in another study, would you?"
)

fig.tight_layout()
fig.savefig(f"{img_dir}again.png")

In [ ]:
""" Aggregate """

all_counts = child_counts.add(adolescent_counts, fill_value=0)
all_counts = all_counts.rename(
    index={
        1.0: "Yes",
        2.0: "Maybe",
        3.0: "No",
        -9.0: "Refused to Answer",
        -100.0: "Missing data",
    }
)

display(
    pd.DataFrame(
        {"Counts": all_counts, "Percentage": 100 * all_counts / all_counts.sum()}
    )
)

In [ ]:
"""
Days of self-reported wear

"""

fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)

child_counts = child_df["smart5_7to9"].value_counts(dropna=False)
adolescent_counts = adolescent_df["smart5_10to17"].value_counts(dropna=False)

# Fill NA with a negative value so it appears at the bottom of my plot
na_val = -1
child_counts.index = child_counts.index.fillna(na_val)
adolescent_counts.index = adolescent_counts.index.fillna(na_val)

child_counts.sort_index(inplace=True)
adolescent_counts.sort_index(inplace=True)

for i, val in child_counts.items():
    axes[0].bar(i, val, color="C0", width=0.5)
for i, val in adolescent_counts.items():
    axes[1].bar(i, val, color="C1", width=0.5)

# Label each bar with the number
for i, v in child_counts.items():
    axes[0].text(i, v + 0.5, str(v), ha="center")
for i, v in adolescent_counts.items():
    axes[1].text(i, v + 0.5, str(v), ha="center")

for axis in axes:
    axis.set_xticks(range(-1, 9), ["N/A", *range(0, 9)])

axes[0].set_title("Children")
axes[1].set_title("Adolescents")

fig.supxlabel("Days of self-reported wear")
fig.supylabel("Count")
fig.suptitle("How many days in total did you wear the smartwatch for?")

fig.tight_layout()
fig.savefig(f"{img_dir}wear.png")

In [ ]:
"""
Aggregate

"""
all_counts = child_counts.add(adolescent_counts, fill_value=0)

fig, axis = plt.subplots(figsize=(10, 5))

for i, val in all_counts.items():
    axis.bar(i, val, width=0.5, color="C0")

# Label each bar with the number
# for i, v in all_counts.items():
#     axis.text(i, v + 0.5, str(int(v)), ha="center")

for axis in axes:
    axis.set_xticks(range(-1, 9), ["N/A", *range(0, 9)])

fig.supxlabel("Days of self-reported wear")
fig.suptitle("How many days in total did you wear the smartwatch for?")

fig.tight_layout()
fig.savefig(f"{img_dir}all_wear.png")

In [ ]:
display(
    pd.DataFrame(
        {"Counts": all_counts, "Percentage": 100 * all_counts / all_counts.sum()}
    )
)

In [ ]:
"""
Cross-check self-reported wear with the smartwatch data

"""
import numpy as np

from analysis_utils import clean

# Read smartwatch data
meal_df = clean.cleaned_smartwatch(keep_catchups=False, keep_day0=True)[
    ["p_id", "delta", "meal_type"]
]

# Keep only the positive entries
meal_df = meal_df[
    meal_df["meal_type"].isin({"Meal", "Drink", "Snack", "No food/drink"})
]

# Convert delta to days only
meal_df["delta"] = meal_df["delta"].dt.days

# Collapse to get one row per participant, showing the number of days they entered on
entries_df = pd.DataFrame(columns=["p_id", "days_entered"], dtype=np.float64)
for i, (p_id, group) in enumerate(meal_df.groupby("p_id")):
    days_entered = set()
    for _, row in group.iterrows():
        days_entered.add(row["delta"])

    entries_df = pd.concat(
        [
            entries_df,
            pd.DataFrame(
                {"p_id": [p_id], "days_entered": [len(days_entered)]}, index=[i]
            ),
        ]
    )

# Combine it with the survey responses
print(qnaire_data.columns)
entries_df = entries_df.merge(
    qnaire_data[["residents_id", "smart5_7to9", "smart5_10to17", "age_dob"]],
    left_on="p_id",
    right_on="residents_id",
    how="left",
)

entries_df["self_reported"] = entries_df["smart5_10to17"].combine_first(
    entries_df["smart5_7to9"]
)

entries_df.head()

In [ ]:
entries_df["self_reported"].value_counts(dropna=False), entries_df["self_reported"].isin({7,8}).sum() / len(entries_df)

In [ ]:
entries_df["days_entered"].value_counts(dropna=False), entries_df["days_entered"].isin({7,8}).sum() / len(entries_df)

In [ ]:
""" Scatter plot where the size corresponds to the number of participants """

fig, axis = plt.subplots(figsize=(5, 5))

n_match = 0
n_less = 0
n_more = 0
for self_reported in entries_df["self_reported"].unique():
    df_slice = entries_df[entries_df["self_reported"] == self_reported]
    for measured, grp in df_slice.groupby("days_entered"):
        axis.scatter(
            self_reported,
            measured,
            s=len(grp) * 15,
            label=self_reported,
            color="grey",
            zorder=4,
        )
        if self_reported == measured:
            n_match += len(grp)
        if self_reported > measured:
            n_less += len(grp)
        if self_reported < measured:
            n_more += len(grp)


axis.plot([0, 8.5], [0, 8.5], "k", linewidth=0.75, zorder=1)

axis.set_xlim(0, 8.5)
axis.set_ylim(0, 8.5)

axis.set_ylabel("Measured")
axis.set_xlabel("Self-reported")

fig.suptitle("Self reported vs Measured Wear Days")

fig.tight_layout()
fig.savefig(f"{img_dir}entry_days_crosscheck.png")

n_match, len(entries_df), n_less, n_more

In [ ]:
""" max length of time """

fig, axis = plt.subplots()

counts = all_df["smart7_7to9"].value_counts(dropna=True).add(
    all_df["smart7_10to17"].value_counts(dropna=True), fill_value=0
)
print(np.sum(counts))

display(
    pd.DataFrame(
        {"Counts": counts, "Percentage": 100 * counts / counts.sum()}
    )
)

# Drop the -99 index
counts = counts.drop(-99)

axis.bar(counts.index, counts.values)

axis.set_ylabel("Count")
axis.set_xlabel("Number of Days")
fig.tight_layout()
fig.savefig(f"{img_dir}n_days.png")

In [ ]:
print(all_df["smart8_7to9"].value_counts(dropna=True))
all_df["smart8_10to17"].value_counts(dropna=True)

In [ ]:
"""
Reasons given for not wearing the smartwatch for the whole study

"""

child_counts = child_df[child_df["smart5_7to9"] != 8]["smart6_7to9"].value_counts(
    dropna=False
)
child_counts.index = child_counts.index.fillna(na_val)

lookup = {
    -9: "Refused to answer",
    1: "Did not like the look of the smartwatch",
    2: "Did not find the smartwatch comfortable to wear",
    3: "Recharging took too much time",
    4: "Did not see the benefit, when could not see the data",
    5: "Other reasons (please specify)",
    na_val: "N/A",
}

print("Children")
for i, val in child_counts.items():
    print("\t", lookup[i], val)

display(
    pd.DataFrame(
        {"Counts": child_counts, "Percentage": 100 * child_counts / child_counts.sum()}
    )
)

In [ ]:
adolescent_counts = adolescent_df[adolescent_df["smart5_10to17"] != 8][
    "smart6_10to17"
].value_counts(dropna=False)
adolescent_counts.index = adolescent_counts.index.fillna(na_val)

lookup = {
    -9: "Refused to answer",
    1: "Did not like the look of the smartwatch",
    2: "Did not find the smartwatch comfortable to wear",
    3: "Recharging took too much time",
    4: "Did not see the benefit, when could not see the data",
    5: "Other reasons (please specify)",
    na_val: "N/A",
}

print("Adolescents")
for i, val in adolescent_counts.items():
    print("\t", lookup[i], val)


# Other reasons
adolescent_df["smart6a_10to17"].value_counts()

Malay                                         | Count | English
----------------------------------------------|-------|-----------------------------------------------
-99                                           | 1     | N/A
Lupa                                          | 1     | "Forget"
Terlupa                                       | 1     | "Forgotten"
terlupa pakai selepas aktiviti merenang       | 1     | "Forgot to use after swimming activity"
pihak sekolah tidak membenarkan memakai gajet | 1     | "The school does not allow the use of gadgets"
Kehabisan bateri                              | 1     | "Out of battery"
Sakit dan gatal                               | 1     | "Pain and itching"

In [ ]:
"""
Labelled bar plots of smartwatch experience out of 5

"""

from textwrap import wrap

fig, axes = plt.subplots(2, 4, figsize=(12, 6))
ticks = tuple(range(-1, 6))
for df, axs, col, colour, text_d in zip(
    [child_df, adolescent_df],
    axes,
    ["smart9_opt*_7to9", "smart_opt*_10to17"],
    ["C0", "C1"],
    [0.1, 0.5],
):
    for i, axis in enumerate(axs, start=1):
        col_name = col.replace("*", str(i))
        counts = df[col_name].value_counts(dropna=False)
        counts.index = counts.index.fillna(-1)

        # Replace Refused to respond
        counts.index = counts.index.where(counts.index != -9, 0)

        axis.bar(counts.index, counts.values, color=colour)

        # Label the bars
        for j, v in counts.items():
            axis.text(j, v + text_d, str(v), ha="center")

        axis.set_xticks(
            ticks,
            ["N/A", "Refused to answer", *range(1, 6)],
            rotation=45,
            ha="right",
        )


titles = [
    "Your child added all their eating occurrences into the smartwatch",
    "Your child was able to enter their food intake correctly into the smartwatch",
    "Your child added all their drinking occurrences into the smartwatch",
    "Your child was able to enter their drink intake correctly into the smartwatch",
    "You added all their eating occurrences into the smartwatch",
    "You were able to enter their food intake correctly into the smartwatch",
    "You added all their drinking occurrences into the smartwatch",
    "You were able to enter their drink intake correctly into the smartwatch",
]
for title, axis in zip(titles, axes.flat):
    axis.set_title("\n".join(wrap(title, 30)))

for axis in axes[0]:
    axis.set_ylim(0, 3)
for axis in axes[1]:
    axis.set_ylim(0, 30)

fig.tight_layout()
fig.savefig(f"{img_dir}experience_5.png")

In [ ]:
""" aggregate """

fig, axes = plt.subplots(1, 4, figsize=(12, 3))
ticks = tuple(range(-1, 6))

for i, axis in enumerate(axes, start=1):
    child_col_name = "smart9_opt*_7to9".replace("*", str(i))
    ad_col_name = "smart_opt*_10to17".replace("*", str(i))
    counts = (adolescent_df[ad_col_name].value_counts(dropna=False)).add(
        child_df[child_col_name].value_counts(dropna=False), fill_value=0
    )
    counts.index = counts.index.fillna(-1)

    # Replace Refused to respond
    counts.index = counts.index.where(counts.index != -9, 0)

    axis.bar(counts.index, counts.values, color="C0")

    # Label the bars
    for j, v in counts.items():
        axis.text(j, v + text_d, str(v), ha="center")

    axis.set_xticks(
        ticks,
        ["N/A", "Refused to answer", *range(1, 6)],
        rotation=45,
        ha="right",
    )


titles = [
    "You added all their eating occurrences into the smartwatch",
    "You were able to enter their food intake correctly into the smartwatch",
    "You added all their drinking occurrences into the smartwatch",
    "You were able to enter their drink intake correctly into the smartwatch",
]
for title, axis in zip(titles, axes):
    axis.set_title("\n".join(wrap(title, 30)))

for axis in axes:
    axis.set_ylim(0, 30)

fig.tight_layout()
fig.savefig(f"{img_dir}experience_5_agg.png")

In [ ]:
"""
Table for charging responses

"""

child_counts = child_df["smart10_7to9"].value_counts(dropna=False)
adolescent_counts = adolescent_df["smart10_10to17"].value_counts(dropna=False)
print(adolescent_counts.sum())
child_counts, adolescent_counts

| Children - charging | Malay   | English | Count |
|---------------------|---------|---------|-------|
|                     | NaN     | NaN     | 1     |
|                     | Senang  | Happy   | 1     |
|                     | Gembira | Happy   | 1     |
|                     | Mudah   | Easy    | 1     |



| Adolescent - charging | Malay                                    | English                             | Count |
|-----------------------|------------------------------------------|-------------------------------------|-------|
|                       | NaN                                      | NaN                                 | 24    |
|                       | -99                                      | Refused to answer                   | 14    |
|                       | Sangat selesa                            | very comfortable                    | 1     |
|                       | Sangat baik                              | very good                           | 1     |
|                       | Sangat mudah                             | very easy                           | 1     |
|                       | best                                     | best                                | 1     |
|                       | Baik                                     | Good                                | 3     |
|                       | Mudah                                    | easy                                | 1     |
|                       | Mudah dicas                              | easy to charge                      | 1     |
|                       | Seronok                                  | Fun                                 | 3     |
|                       | Senang                                   | Happy                               | 2     |
|                       | senang                                   | happy                               | 2     |
|                       | Gembira, sebab boleh menggunakannya esok | happy because i can use it tomorrow | 1     |
|                       | mudah                                    | easy                                | 2     |
|                       | SELESA                                   | comfortable                         | 1     |
|                       | Selesa                                   | comfortable                         | 1     |
|                       | baik                                     | good                                | 1     |
|                       | Baik dan mudah                           | good and easy                       | 1     |
|                       | Agak mudah                               | quite easy                          | 1     |
|                       | Ok                                       | ok                                  | 5     |
|                       | Okey                                     | ok                                  | 1     |
|                       | Okey je                                  | it's ok                             | 1     |
|                       | OK                                       | ok                                  | 1     |
|                       | Okay je                                  | it's ok                             | 1     |
|                       | okay                                     | ok                                  | 1     |
|                       | Biasa                                    | normal                              | 1     |
|                       | Biasa saja                               | it's normal                         | 1     |
|                       | tiada perasaan                           | no feelings                         | 1     |
|                       | Tiada                                    | none                                | 1     |
|                       | Tak selesa                               | uncomfortable                       | 1     |
|                       | Susah sikit                              | a little difficult                  | 1     |

In [ ]:
"""
Table for any other comments

"""

parent_counts = child_df["smart11a_7to9"].value_counts(dropna=False)
child_counts = child_df["smart11b_7to9"].value_counts(dropna=False)
adolescent_counts = adolescent_df["smart11_10to17"].value_counts(dropna=False)

print(parent_counts, child_counts, adolescent_counts, sep="\n\n")


| Adolescents | Malay                                                                                                                                                 | English                                                                                                                                    | Count |
|-------------|-------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------|-------|
|             | -99                                                                                                                                                   | -99                                                                                                                                        | 54    |
|             | NaN                                                                                                                                                   | NaN                                                                                                                                        | 16    |
|             | -                                                                                                                                                     | -                                                                                                                                          | 1     |
|             | 5                                                                                                                                                     | 5                                                                                                                                          | 1     |
|             | 6                                                                                                                                                     | 6                                                                                                                                          | 1     |
|             | Selesa                                                                                                                                                | Comfortable                                                                                                                                | 1     |
|             | Seronok                                                                                                                                               | It's fun                                                                                                                                   | 1     |
|             | tiada                                                                                                                                                 | no                                                                                                                                         | 1     |
|             | tidak selesa dipakai kerana memakainya dalam jangka masa yang lama                                                                                    | uncomfortable to wear because of wearing it for a long time                                                                                | 1     |
|             | Saya rasa sungguh risau apabila memakai jam aktiviti monitor ketika tidur kerana takut akan merosakkan jam tersebut akibat terketuk pada tiang katil. | I feel really worried when wearing the monitor activity clock while sleeping for fear of damaging the clock due to knocking on the bedpost | 1     |

In [ ]:
"""
Find out if participants also found the AX6 uncomfortable

"""

ax6_df = qnaire_data[qnaire_data["activitymonitorwilling"] == 1].copy()

lims = (7, 9, 18)
ax6_df["child"] = (ax6_df["age_dob"] >= lims[0]) & (ax6_df["age_dob"] <= lims[1])
ax6_df["adolescent"] = (ax6_df["age_dob"] > lims[1]) & (ax6_df["age_dob"] <= lims[2])

# Check no overlap
assert not (ax6_df["child"] & ax6_df["adolescent"]).any()

# Check no unassigned
assert (ax6_df["child"] | ax6_df["adolescent"]).all()

len(ax6_df), ax6_df["child"].sum(), ax6_df["adolescent"].sum()

In [31]:
child_ax6 = ax6_df[ax6_df["child"]].copy()
adolescent_ax6 = ax6_df[ax6_df["adolescent"]].copy()

In [ ]:
# Comfort is 2

print(child_ax6["fdbck5_7to9"].value_counts())
print(adolescent_ax6["fdbck5_10to17"].value_counts())

In [ ]:
# Find the proportion of respondents who found the AX6 uncomfortable
all_codes = {1.0, 2.0, 3.0, 4.0}
comfort_code = 2.0
print(
    (100 * (child_ax6["fdbck5_7to9"] == comfort_code)).sum()
    / (child_ax6["fdbck5_7to9"].isin(all_codes).sum())
)
print(
    (100 * (adolescent_ax6["fdbck5_10to17"] == comfort_code)).sum()
    / (adolescent_ax6["fdbck5_10to17"].isin(all_codes).sum())
)